<a href="https://www.kaggle.com/code/dzung271828/ouro-trace?scriptVersionId=288557226" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Setup libraries

In [1]:
!uv pip install --upgrade pip
!uv pip uninstall transformers tokenizers accelerate -q

!uv pip install "transformers==4.56.0" "protobuf==5.29.3" -q
!uv pip install torch datasets -q
!uv pip install pandas matplotlib seaborn tqdm wandb pyyaml
!uv pip install bitsandbytes accelerate optimum lm_eval
# !uv pip install -r requirements.txt
!uv pip install --force-reinstall --no-cache-dir "numpy<2.0"

Using Python 3.12.12 environment at: /usr
Resolved 1 package in 180ms
Prepared 1 package in 664ms
Uninstalled 1 package in 307ms
Installed 1 package in 102ms
 - pip==24.1.2
 + pip==25.3
Using Python 3.12.12 environment at: /usr
Audited 6 packages in 206ms
Using Python 3.12.12 environment at: /usr
Resolved 95 packages in 2.92s
Prepared 21 packages in 3.38s
Installed 21 packages in 692ms
 + accelerate==1.12.0
 + bitsandbytes==0.49.0
 + colorama==0.4.6
 + dataproperty==1.1.0
 + evaluate==0.4.6
 + jsonlines==4.0.0
 + lm-eval==0.4.9.2
 + mbstrdecoder==1.1.4
 + optimum==2.1.0
 + pathvalidate==3.3.1
 + portalocker==3.2.0
 + pybind11==3.0.1
 + pytablewriter==1.2.1
 + rouge-score==0.1.2
 + sacrebleu==2.5.1
 + sqlitedict==2.1.0
 + tabledata==1.3.4
 + tcolorpy==0.1.7
 + tqdm-multiprocess==0.0.11
 + typepy==1.3.4
 + word2number==1.1
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 109ms
Prepared 1 package in 791ms
Uninstalled 1 package in 67ms
Installed 1 package in 33ms
 - numpy==2

# Suppress warnings

In [8]:
# Suppress warnings for clean output
import warnings
import os

warnings.filterwarnings("ignore", category=UserWarning)
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
print("✅ Packages installed successfully!")

✅ Packages installed successfully!


# Install Libraries

In [9]:
"Built-in libraries"
import re
import sys
import gc
import time
import json
import hashlib
import glob
import zipfile
from io import StringIO
from datetime import datetime
from typing import Dict, List, Optional, Tuple, Any
import yaml
import logging
import random

"Deep learning and NLP libraries"
import torch
import torch.nn.functional as F
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    GenerationConfig,
    logging as hf_logging,
)

"Data processing libraries"
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import wandb
from tqdm.auto import tqdm
from IPython import get_ipython

# Configure logging
logging.getLogger("ContinuousBatchingLogger").setLevel(logging.ERROR)
hf_logging.set_verbosity_error()


print(f"Python Version: {sys.version}")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
!nvidia-smi

Python Version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
PyTorch Version: 2.9.0+cu126
CUDA Available: True
CUDA Version: 12.6
Fri Dec 26 15:31:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|   

In [10]:
import os


def configure_environment_paths():
    """Detect environment and configure paths"""
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"

    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")

    return base_data_path, base_output_path, environment_name


INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()

✅ Environment: Google Colab
📂 Data Path: /content/
📦 Output Path: /content/


# Setup WANDB

In [11]:
import os
import wandb

if "colab" in ENV_NAME:
    from google.colab import userdata

    try:
        # Ensure 'WANDB_API_KEY' is the exact name in your Colab Secrets (the key icon)
        wandb_key = userdata.get("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Colab Secrets: {e}")

# 2. Check if running in Kaggle
elif "kaggle" in ENV_NAME:
    try:
        from kaggle_secrets import UserSecretsClient

        user_secrets = UserSecretsClient()
        wandb_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=wandb_key)
    except Exception as e:
        print(f"Could not retrieve W&B API key from Kaggle Secrets: {e}")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Config input/output path and clone latest repo

In [12]:
# Clone the latest github repo version
%cd {OUTPUT_PATH}
torch.cuda.empty_cache()
!rm -rf OuroTrace

/content


In [13]:
!git clone --branch claude https://github.com/dzungphieuluuky/OuroTrace.git
%cd OuroTrace

Cloning into 'OuroTrace'...
remote: Enumerating objects: 2354, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 2354 (delta 137), reused 136 (delta 47), pack-reused 2118 (from 2)
Receiving objects: 100% (2354/2354), 7.07 MiB | 14.49 MiB/s, done.
Resolving deltas: 100% (1494/1494), done.
/content/OuroTrace


# Run Benchmark

In [ ]:
from src.config_loader import load_config_from_json, post_process_config
from src.new_runner import run_experiment
from src.evaluation_analysis import analyze_experiment_results

def set_all_seeds(seed):
    random.seed(seed)                          # Python random
    os.environ['PYTHONHASHSEED'] = str(seed)  # Python hash seed
    np.random.seed(seed)                      # NumPy
    torch.manual_seed(seed)                   # PyTorch CPU & GPU

    # Additional GPU-specific settings
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)      # For multi-GPU

set_all_seeds(1415)
# 1. Load Configuration from JSON
config = load_config_from_json("configs/ouro_1.4b_thinking.json")

# 2. Post-process (Convert 'torch.float16' string to object, generate timestamps)
config = post_process_config(config)

config["INFERENCE_STEPS"] = [5]
config["OPTIMIZATION"]["enable_batch"] = False
# config["EVAL_SETTINGS"]["calculate_perplexity"] = False
# config["DATA"]["n_ary"]["num_samples_per_level"] = 0
# config["DATA"]["p_hop"]["num_samples_per_level"] = 0
# config["DATA"]["igsm"]["num_samples"] = 0
# config["DATA"]["reasoning_primitives"]["num_samples"] = 0
# config["ENABLE_HEAVY_BENCHMARKS"] = True
# 4. Execute
print("🚀 Starting Experiment...")
try:
    del model, tokenizer
    torch.cuda.empty_cache()
    gc.collect()
except:
    pass

try:
    simple_reasoning_results, ppl_results, primitives_results, benchmark_results = run_experiment(config)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

🚀 Starting Experiment...
Initializing W&B (timeout: 30s)...


W&B initialized successfully

EXPERIMENT CONFIGURATION
Model Path: ByteDance/Ouro-1.4B-Thinking
UT Steps to Test: [5]
Data Type: torch.bfloat16
4-bit Quantization: False
Torch Compile: False
Max Batch Size: 8
Max New Tokens: 16
Batching: False
Calculate Perplexity: True
Early Exit: 1.0

Quality monitor initialized:
    Garbage threshold: 30%
    Example similarity threshold: 85%
    Min samples before check: 10
Random seed set to 42

LOADING TEST DATASETS
Generating new test datasets...
Generated test datasets successfully

Dataset Summary:
   n_ary       :  500 samples
   p_hop       :  300 samples
   igsm        :  100 samples


PAPER COMPLIANCE CHECK
Task Alignment: {'has_n_ary': True, 'has_p_hop': True, 'has_igsm': True, 'all_paper_tasks': True}
UT Steps Coverage: {'min_ut': 5, 'max_ut': 5, 'covers_baseline': False, 'covers_paper_range': False, 'recommended_range': [1, 2, 4, 8]}

Preparing perplexity evaluation data...
Prepared 50 samples for PPL

✅ Configuration saved to ../result

config.json: 0.00B [00:00, ?B/s]

configuration_ouro.py: 0.00B [00:00, ?B/s]


Base config loaded
   Original UT steps: 4
   Original early exit: 1.0

Modified config:
   New UT steps: 5
   Early exit threshold: 1.0 (from default)


...


────────────────────────────────────────────────────────────
APPLYING SAFE OPTIMIZATIONS
────────────────────────────────────────────────────────────
   Flash Attention / SDPA enabled
   TF32 enabled for matmul
   cuDNN auto-tuning enabled
   Memory pool optimized
   Running 3 warmup passes...
   Warmup complete
────────────────────────────────────────────────────────────

MODEL LOADED SUCCESSFULLY
Device: cuda:0
Model dtype: torch.bfloat16
VERIFIED UT steps: 5
VERIFIED early exit: 1.0

Building task templates...
Task templates with pre-tokenized components computed.
    System prompt N_ary tokens: 813 tokens
    System prompt P_hop tokens: 682 tokens
    System prompt IGSM tokens: 1136 tokens
    User prefix tokens: 14 tokens
    User suffix tokens: 6 tokens
    Force start tokens: 4 tokens
Task templates built successfully

✅ Configuration saved to ../results_20251226_153141_UT_5/config.json
✅ Task templates saved to ../results_20251226_153141_UT_5/task_templates.json
Experiment con

Calculating PPL (UT=5):   0%|          | 0/8 [00:00<?, ?it/s]


Perplexity Results:
   Perplexity: 0.2358
   Avg Loss:   1.2660

ACCURACY EVALUATION


──────────────────────────────────────────────────────────────────────
Task: N_ARY
──────────────────────────────────────────────────────────────────────
Total Samples: 500
Batch Size: 1 (Sequential)
Strategy: Sequential Processing

Batch size < 1 or not enough items, processing sequentially.
Processing 500 items sequentially...


   n_ary:   0%|          | 0/500 [00:00<?, ?it/s]

    test_input        full_response  generated_tokens
0  871 + 556 =  [FINAL]  1427 [END]                 8
    test_input        full_response  generated_tokens
0  888 + 759 =  [FINAL]  1647 [END]                 8
    test_input        full_response  generated_tokens
0  670 + 418 =  [FINAL]  1088 [END]                 8
    test_input        full_response  generated_tokens
0  661 + 544 =  [FINAL]  1205 [END]                 8
    test_input        full_response  generated_tokens
0  328 + 902 =  [FINAL]  1230 [END]                 8
    test_input       full_response  generated_tokens
0  696 + 243 =  [FINAL]  939 [END]                 7
    test_input        full_response  generated_tokens
0  810 + 357 =  [FINAL]  1167 [END]                 8
    test_input       full_response  generated_tokens
0  191 + 780 =  [FINAL]  971 [END]                 7
    test_input        full_response  generated_tokens
0  933 + 652 =  [FINAL]  1585 [END]                 8
    test_input        full_respo

In [ ]:
import os
import glob
import zipfile
from typing import List
def find_result_folders(base_path: str) -> List[str]:
    """
    Return a list of absolute paths to all directories under `base_path`
    whose names start with 'results_'.
    """
    pattern = os.path.join(base_path, "results_*")
    # glob returns both files and directories; filter to directories only
    return [p for p in glob.glob(pattern) if os.path.isdir(p)]
def zip_folder(folder_path: str, output_base_path: str) -> bool:
    """
    Zip the contents of `folder_path` into a file named
    <folder_name>.zip` inside `output_base_path`.

    Returns True on success, False otherwise.
    """
    folder_name = os.path.basename(folder_path)
    zip_path = os.path.join(output_base_path, f"{folder_name}.zip")
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(
            zip_path, mode="w", compression=zipfile.ZIP_DEFLATED
        ) as zipf:
            for root, _, files in os.walk(folder_path):
                for file in files:
                    full_path = os.path.join(root, file)
                    # Preserve relative path inside the zip
                    arcname = os.path.relpath(full_path, os.path.dirname(folder_path))
                    zipf.write(full_path, arcname)
        print(f"   ✅ Created ZIP: {os.path.basename(zip_path)}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False
def zip_stats_results_folders(output_base_path: str) -> None:
    """
    Main driver: locate all result folders and zip each one.
    """
    # Ensure the output directory exists
    os.makedirs(output_base_path, exist_ok=True)
    result_folders = find_result_folders(output_base_path)
    if not result_folders:
        print(f"⚠️ No folders starting with 'results_' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = 0
    for folder in result_folders:
        if zip_folder(folder, output_base_path):
            successful += 1
    print(
        f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s)."
    )
if __name__ == "__main__":
    try:
        # Prefer an environment variable; fall back to a global if defined
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        # The script expects a sub‑folder named 'OuroTrace' under OUTPUT_PATH
        target_path = os.path.join(output_root, "")
        zip_stats_results_folders(target_path)
    except Exception as e:
        print(f"❌ An error occurred: {e}")

In [ ]:
df_simple = pd.DataFrame(simple_reasoning_results)
df_ppl = pd.DataFrame(ppl_results)

print("Final Inspection:\n")
print("Top 20 Accuracy Report:\n")
print(df_simple.head(20))
print(f"Full Response:\n")
print(df_simple["full_response"])
print("Perplexity Report:\n")
print(df_ppl.head(20))

In [ ]:
print(df_simple[["full_response", "generated_tokens"]])